In [1]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
Number of GPUs available: 2
GPU 0: NVIDIA GeForce RTX 2070
GPU 1: NVIDIA GeForce RTX 2070
GPU 0 memory allocated: 0.00 GB
GPU 0 memory reserved: 0.00 GB
GPU 1 memory allocated: 0.00 GB
GPU 1 memory reserved: 0.00 GB


In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2H5, FFF2H5

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [ ]:
# Initialize Parameters
scope = ScopeClass() 
data = DataContainer() # you can also initialize these with parameters, but it is not necessary due to defaults
settings = Settings(name='demo', connection_config_location=None) # you also must give a name for the analysis your are doing
experiment = Experiment(initial_data_location=r'smFISH_images\Eric_smFISH_images\20230511\DUSP1_DexTimeConcSweep_10nM_75min_041223')

settings.load_in_mask = True
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.voxel_size_z = 500

settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90] # most of these options can be done for individually cyto and nuc segmentation, 
                                        # and a list can be or a single float can be passed for both
                                        # always in the order cyto, nuc
settings.cellpose_pretrained_model = [r"GAPDH_cyto", r'DAPI_nuclei'] 

In [ ]:
# You can check that all the manditory parameters are set by calling validate
Parameters.validate()

In [ ]:
FFF2H5()
SimpleCellposeSegmentaion()
BIGFISH_SpotDetection()
Save_Masks()
Save_Parameters()
Save_Outputs()
return_to_NAS()

In [6]:
pipeline = Pipeline()

In [ ]:
pipeline.run_on_cluster('/home/formanj/Github/AngleFISH/cluster', name='Demo')